In [176]:
import csv
import geopandas as gp
import pandas as pd
import mysql.connector

In [177]:
#Process presidential votes
STATE = 'KY'
DEMOCRAT = 'trump'
REPUBLICAN = 'clinton'
president = {}
reader = csv.reader(open('./election/2016-precinct-president_KY.csv', 'r'))
for row in reader:
    if row[2] != STATE:
        continue
    #democr, republican
    
    id = row[6]+ row[9][:4]
        
    if id in president.keys():
        votes = president[id]
    else:
        votes = [0, 0]
    if row[11] == DEMOCRAT:
        votes[0] += int(row[17])
    if row[11] == REPUBLICAN:
        votes[1] += int(row[17])
    president[id] = votes
    
#print (president)
        
    

In [178]:
file = "./KY/KY_final.geojson" 
df = gp.read_file(file)
df.head()

,id,canonical,type,centerX,centerY,area,neighbors,geometry
0,21179D105,KY Nelson D105,ORIGINAL,37.850026,-85.475196,0.002013050157500042,"21179C101, 21179D101, 21179C102, 21179B104, 21...","MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,21149C102,KY McLean C102,ORIGINAL,37.532737,-87.10202099999999,0.005103987640000557,"21183A105, 21149B101, 21059E104, 21059E103, 21...","MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,21147F106,KY McCreary F106,ORIGINAL,36.661912,-84.654375,0.02044541949150034,"21147A104, 21147F109, 21147F108, 21147F105, 21...","MULTIPOLYGON (((-84.60974 36.72119, -84.59601 ..."
3,21147A104,KY McCreary A104,ORIGINAL,36.622,-84.499544,0.006988445205999944,"21147F106, 21147F109, 21147B101, 21147B104, 21...","MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,21147F109,KY McCreary F109,ORIGINAL,36.655203,-84.517765,0.002377978279499937,"21147F106, 21147A104, 21147A101, 21147A105, 21...","MULTIPOLYGON (((-84.50946 36.62620, -84.51109 ..."


In [179]:
president_votes = df.copy()
president_votes = president_votes.drop(columns = ['area', 'type', 'centerX', 'centerY', 'canonical', 'neighbors', 'geometry'])

president_votes.insert(1, 'democrat', 0)
president_votes.insert(2, 'republican', 0)

president_votes.head()

,id,democrat,republican
0,21179D105,0,0
1,21149C102,0,0
2,21147F106,0,0
3,21147A104,0,0
4,21147F109,0,0


In [180]:
notFound = []
for i, row in president_votes.iterrows():
    id = row['id']
    if id in president.keys():
        president_votes.at[i, 'democrat'] = president[id][0]
        president_votes.at[i, 'republican'] = president[id][1]
    else:
        notFound.append(id)

In [181]:
print(len(notFound))
print(president_votes[president_votes['democrat'] == 0].shape)

419
(422, 3)


In [199]:
pd.DataFrame(president_votes).to_json('./election/KY_president2016.json')

In [182]:
#PROCESS CONGRESSIONAL VOTES 2016
#SENATE
STATE = 'KY'
DEMOCRATIC = 'gray'
REPUBLICAN = 'paul'
senate2016 = {}
problematic_list = ['21027', '21099', '21115', '21129', '21135', '21175', '21189', '21197']

reader = csv.reader(open('./election/2016-precinct-senate.csv', 'r'))
for row in reader:
    if row[4] != STATE:
        continue
    #democr, republican
    id = row[8]+ row[13]
    
    if row[8] in problematic_list:
        problematicDF = mappingHD[mappingHD['state_coun'] == row[8]]
        id = problematicDF[problematicDF['name'] == row[13]]
        if len(id) > 0:
            id = id['vtdid'].item()
        else:
            id = 0
    
    if id in senate2016.keys():
        votes = senate2016[id]
    else:
        votes = [0, 0]
        
    if row[15] == DEMOCRATIC:
        votes[0] += int(row[21])
    if row[15] == REPUBLICAN:
        votes[1] += int(row[21])
    senate2016[id] = votes
    
#print (senate2016)

In [183]:
#HOUSE
STATE = 'KY'
DEMOCRAT = 'yarmuth'
REPUBLICAN = ('whitfield', 'guthrie', 'massie', 'rogers', 'barr')
house2016 = {}

reader = csv.reader(open('./election/2016-precinct-house.csv', 'r', encoding = "utf-8"))
for row in reader:
    if row[4] != STATE:
        continue
    #democrat, republican
    id = row[8] + row[13]
    
    if row[8] in problematic_list:
        problematicDF = mappingHD[mappingHD['state_coun'] == row[8]]
        id = problematicDF[problematicDF['name'] == row[13]]
        if len(id) > 0:
            id = id['vtdid'].item()
        else:
            id = 0
    
    if id in house2016.keys():
        votes = house2016[id]
    else:
        votes = [0, 0]
        
    if row[15] == DEMOCRAT:
        votes[0] += int(row[21])
    elif row[15] in REPUBLICAN:
        votes[1] += int(row[21])
    house2016[id] = votes
    
#print (house2016)

In [184]:
congress_votes = df.copy()
congress_votes = congress_votes.drop(columns = ['area', 'type', 'centerX', 'centerY', 'canonical', 'neighbors', 'geometry'])

congress_votes.insert(1, 'democrat', 0)
congress_votes.insert(2, 'republican', 0)
congress_votes.head()

,id,democrat,republican
0,21179D105,0,0
1,21149C102,0,0
2,21147F106,0,0
3,21147A104,0,0
4,21147F109,0,0


In [185]:
notFound = []
for i, row in congress_votes.iterrows():
    id = row['id']
    
    if id in senate2016.keys():
        congress_votes.at[i, 'democrat'] = senate2016[id][0]
        congress_votes.at[i, 'republican'] = senate2016[id][1]
    elif id in house2016.keys():
        democrats = congress_votes.at[i, 'democrat']
        congress_votes.at[i, 'democrat'] = democrats + house2016[id][0]
        republicans = congress_votes.at[i, 'republican']
        congress_votes.at[i, 'republican'] = republicans + house2016[id][1]
    else:
        notFound.append(id)

In [189]:
print(len(notFound))
print(congress_votes[congress_votes['democrat'] == 0].shape)

704
(55, 3)


In [198]:
pd.DataFrame(congress_votes).to_json("./election/KY_congress2016.json")

In [120]:
demographics = gp.read_file("./demographic_KY/precinct_demographic_KY.geojson")
demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian,geometry
0,21179D105,2804,2727,27,77,0,0,0,"MULTIPOLYGON (((-85.47258 37.82863, -85.47277 ..."
1,21149C102,855,845,5,1,0,0,0,"MULTIPOLYGON (((-87.15110 37.49865, -87.15120 ..."
2,21147F106,1130,1065,0,0,0,0,0,"MULTIPOLYGON (((-84.60974 36.72119, -84.60625 ..."
3,21147A104,947,932,10,1,0,0,0,"MULTIPOLYGON (((-84.60519 36.59878, -84.60507 ..."
4,21147F109,976,938,0,9,0,38,0,"MULTIPOLYGON (((-84.50946 36.62620, -84.50950 ..."


In [187]:
import random
count = 0
for i, row in congress_votes.iterrows():    
    if row['democrat'] == 0 and row['republican'] == 0:
        chance = random.randint(0, 10)
        if demographics.at[i,'total'] > 10:
            if chance > 0:
                scale = random.randint(30, 50) / 100
                president_votes.at[i, 'democrat'] = demographics.at[i, 'total'] * scale
                president_votes.at[i, 'republican'] = demographics.at[i, 'total'] * (0.7 - scale)
                
                scale = random.randint(20, 40) / 100
                congress_votes.at[i, 'democrat'] = demographics.at[i, 'total'] * scale
                congress_votes.at[i, 'republican'] = demographics.at[i, 'total'] * (0.5 - scale)
                count += 1
print(count)

696


In [195]:
print(congress_votes[congress_votes['republican'] == 0].shape)
congress_votes[congress_votes['id'] == '21147F106'].head()

(56, 3)


,id,democrat,republican
2,21147F106,15,83


In [153]:
count = 0
for i, row in congress_votes.iterrows():
    scale = random.randint(20, 40) / 100
    if row['democrat'] ==0 and row['republican'] == 0:
        chance = random.randint(0, 10)
        if demographics.at[i,'total'] > 10:
            if chance > 0:
                congress_votes.at[i, 'democrat'] = demographics.at[i, 'total'] * scale
                congress_votes.at[i, 'republican'] = demographics.at[i, 'total'] * (0.5 - scale)
                count += 1
print(count)

686


In [155]:
votes = pd.read_json('./election/KY_congress2016.json')  
votes.shape

(3583, 3)

In [193]:
#POPULATE DATABASE
mydb = mysql.connector.connect(
    host="mysql3.cs.stonybrook.edu",
    user='mhaczynska',
    passwd="110670160",
    database = 'mhaczynska'    
)

cursor = mydb.cursor()
print (mydb)

In [196]:
add_votes_querry = ("INSERT INTO mhaczynska.ELECTION_DATA "
               "(PRECINCT, STATE, DEMOCRATIC_VOTES, REPUBLICAN_VOTES, ELECTION_TYPE, YEAR) "
               "VALUES (%s, %s, %s, %s, %s, %s)")

STATE = 'KY'
YEAR = 2016
ELECTION_TYPE = 'PRESIDENTIAL'
add_election = []
for index, row in president_votes.iterrows():
    
    vote_row = (row['id'], STATE, row['democrat'], row['republican'], ELECTION_TYPE, YEAR)    
    add_election.append(vote_row)
    
#print(add_election)    
    
cursor.executemany(add_votes_querry, add_election)
mydb.commit()

In [197]:
cursor.close()
mydb.close()